In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv()
model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description='Sentiment of the review')

class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='Issue category')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='Emotional tone')
    urgency: Literal["low", "medium", "high"] = Field(description='Urgency level')

sentiment_model = model.with_structured_output(SentimentSchema)
diagnosis_model = model.with_structured_output(DiagnosisSchema)

In [ ]:
class ReviewState(TypedDict):
    review: str
    sentiment: str
    diagnosis: dict
    response: str

In [ ]:
def find_sentiment(state: ReviewState):
    prompt = f'Find sentiment of the following review: \n{state["review"]}'
    sentiment = sentiment_model.invoke(prompt).sentiment
    return {'sentiment': sentiment}

def positive_response(state: ReviewState):
    prompt = f"Write a warm thank-you response for this positive review: \n'{state['review']}'"
    response = model.invoke(prompt).content
    return {'response': response}

def run_diagnosis(state: ReviewState):
    prompt = f"Diagnose the following negative review: \n{state['review']}"
    diagnosis = diagnosis_model.invoke(prompt)
    return {'diagnosis': diagnosis.model_dump()}

def negative_response(state: ReviewState):
    d = state['diagnosis']
    prompt = f"""You are a support assistant. Write an empathetic, helpful resolution message.
    User issue: {d['issue_type']}
    User tone: {d['tone']}
    Urgency: {d['urgency']}
    Original Review: {state['review']}"""
    response = model.invoke(prompt).content
    return {'response': response}

In [ ]:
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:
    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [ ]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_sentiment)

graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()

In [ ]:
neg_review = "The app crashed three times today. Very frustrating experience."
result = workflow.invoke({'review': neg_review})
import pprint
pprint.pprint(result)